In [1]:
# Importing the libraries
import numpy as np
import pandas as pd

In [2]:
# Reading the dataset
dataset = pd.read_csv('Downloads/archive/suppliers_data.csv')
dataset_copy = dataset.copy()

In [3]:
dataset.head()

,inv_id,supplier,name,address,city,zip,state,country,latitude,longitude
0,465224,supplier1,#2 Hastings Street,2 Hastings Street,Noosa,4567,QL,Australia,-26.38620,153.08700
1,510663,supplier1,'A Alera Room and Breakfast,Via Roma 10-12,Malfa,98050,NaN,Italy,38.58050,14.83880
2,551357,supplier1,'Lindian Myth' Sea View Studios,Krana,Rhodes,85107,NaN,Greece,36.08800,28.08000
3,394967,supplier1,'S Hertogenmolens,Demerstraat 1,Aarschot,3200,NaN,Belgium,50.98640,4.82882
4,535087,supplier1,*Hotel Punta Diamante Business and Premium Res...,Lote 1 Mediterrane,Bucaramanga,NaN,NaN,Colombia,7.02252,-73.08300


In [4]:
# First: we convert the names of the hotels in lowercase
dataset_copy["name"] = [x.lower() for x in dataset_copy["name"]]

In [5]:
# Second: we split the names of the hotels on spaces
dataset_copy["name"] = [x.split() for x in dataset_copy["name"]]

In [6]:
# Third: we erase the short list of words we consider not significant
# We put these words in a file named 'stopwords.txt'
# Going on with the work, we can change this file, based on what we observe
stopwords = set(w.rstrip() for w in open("Downloads/archive/hotel_stopwords.txt"))
for w in stopwords:
    for i in range(len(dataset_copy)):
        if w in dataset_copy.iloc[i,2]: dataset_copy.iloc[i,2].remove(w)

In [7]:
# Forth: now let's concatenate the words in the Hotel names
# It is gonna make what we do next way easier
dataset_copy["name"] = [" ".join(x) for x in dataset_copy["name"]]

In [8]:
# Fifth: let's make the mapping dataframe
length_dataset = len(dataset_copy)
mapping = pd.DataFrame(columns=['hq_id','inv_id','inv_name'])

# We simply sort the dataframe by name
# So, same hotels are gonna be consecutive
dataset_copy = dataset_copy.sort_values(by=['name'])
b = 0
e = 0
while (e < length_dataset-1):
    if (dataset_copy.iloc[e,2] == dataset_copy.iloc[e+1,2]):
        e += 1
    else:
        if (b != e):
            for i in range(b,e+1):
                if (dataset_copy.iloc[i,1] == "supplier1"):
                    hq_id = int(dataset_copy.iloc[i,0])
                    mapping.loc[len(mapping)]=[hq_id,hq_id,"supplier1"] 
            for i in range(b,e+1):
                if (dataset_copy.iloc[i,1] != "supplier1"):
                    mapping.loc[len(mapping)] = [hq_id, int(dataset_copy.iloc[i,0]), dataset_copy.iloc[i,1]]
        b = e + 1   
        e = e + 1        

In [9]:
mapping.head()

,hq_id,inv_id,inv_name
0,497740,497740,supplier1
1,497740,253112,supplier2
2,455983,455983,supplier1
3,455983,235451,supplier2
4,606740,606740,supplier1


In [12]:
# Finally, let's save the mapping dataset as "mapping.csv"
mapping.to_csv('Downloads/archive/mapping.csv', sep=',')

In [14]:
dataset[dataset["inv_id"] == 497740]["name"]

1023    A & J Budget Hotel
Name: name, dtype: object

In [15]:
dataset[dataset["inv_id"] == 253112]["name"]

66354    A & J Budget Hotel
Name: name, dtype: object

In [16]:
# Easy Peasy!